In [ ]:
!pip -q install --upgrade selenium beautifulsoup4 python-dotenv openai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 84.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 930.8/930.8 kB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 2.6 MB/s eta 0:00:00


In [ ]:
import os, io, json, zipfile, urllib.request, stat

BASE = "/usr/local"
CHROME_DIR = os.path.join(BASE, "chrome-linux64")
DRIVER_DIR = os.path.join(BASE, "chromedriver-linux64")
CHROME_BIN = os.path.join(CHROME_DIR, "chrome")
DRIVER_BIN = os.path.join(DRIVER_DIR, "chromedriver")

def ensure_chrome_and_driver():
    if os.path.exists(CHROME_BIN) and os.path.exists(DRIVER_BIN):
        print("Chrome + Chromedriver already present.")
        return
    url = "https://googlechromelabs.github.io/chrome-for-testing/known-good-versions-with-downloads.json"
    with urllib.request.urlopen(url) as r:
        data = json.load(r)
    latest = data["versions"][-1]
    chrome_zip = [d["url"] for d in latest["downloads"]["chrome"] if d["platform"]=="linux64"][0]
    driver_zip = [d["url"] for d in latest["downloads"]["chromedriver"] if d["platform"]=="linux64"][0]

    def dl_and_extract(zurl):
        with urllib.request.urlopen(zurl) as resp:
            zf = zipfile.ZipFile(io.BytesIO(resp.read()))
            zf.extractall(BASE)

    print("Downloading Chrome:", chrome_zip)
    dl_and_extract(chrome_zip)
    print("Downloading Chromedriver:", driver_zip)
    dl_and_extract(driver_zip)

    os.chmod(CHROME_BIN, os.stat(CHROME_BIN).st_mode | stat.S_IEXEC)
    os.chmod(DRIVER_BIN, os.stat(DRIVER_BIN).st_mode | stat.S_IEXEC)
    print("Ready:", CHROME_BIN, "|", DRIVER_BIN)

ensure_chrome_and_driver()


Ready: /usr/local/chrome-linux64/chrome | /usr/local/chromedriver-linux64/chromedriver


In [ ]:
# --- Attach Mode v3: fix crashpad perms + disable crashpad + attach ---
import os, subprocess, shutil, time, uuid, stat

CHROME_DIR = "/usr/local/chrome-linux64"
CHROME_BIN = f"{CHROME_DIR}/chrome"
DRIVER_BIN = "/usr/local/chromedriver-linux64/chromedriver"

assert os.path.exists(CHROME_BIN),  f"Chrome binary not found at {CHROME_BIN}"
assert os.path.exists(DRIVER_BIN),  f"Chromedriver not found at {DRIVER_BIN}"

# 0) Kill leftovers
for proc in ("chrome", "chromium", "chromedriver"):
    subprocess.run(["pkill","-f",proc], check=False, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

# 1) Fix permissions on crashpad + helpers (prevents 'Permission denied' on spawn)
for helper in ("chrome_crashpad_handler", "chrome_sandbox", "nacl_helper", "nacl_helper_bootstrap"):
    p = os.path.join(CHROME_DIR, helper)
    if os.path.exists(p):
        os.chmod(p, os.stat(p).st_mode | stat.S_IEXEC)
# main binary too
os.chmod(CHROME_BIN, os.stat(CHROME_BIN).st_mode | stat.S_IEXEC)

# 2) Fresh HOME + fresh profile (under /mnt/data to avoid /tmp churn)
home_dir    = f"/mnt/data/home-{uuid.uuid4().hex}"
profile_dir = f"/mnt/data/chrome-prof-{uuid.uuid4().hex}"
os.makedirs(home_dir, exist_ok=True)
os.makedirs(profile_dir, exist_ok=True)
os.environ["HOME"] = home_dir

# 3) Launch Chrome (let it choose a port; write DevToolsActivePort)
chrome_cmd = [
    CHROME_BIN,
    "--headless=new",
    f"--user-data-dir={profile_dir}",
    "--no-sandbox",
    "--disable-dev-shm-usage",
    "--disable-gpu",
    "--disable-extensions",
    "--disable-notifications",
    "--no-first-run",
    "--no-default-browser-check",
    "--disable-background-networking",
    "--disable-sync",
    "--metrics-recording-only",
    "--safebrowsing-disable-auto-update",
    "--window-size=1280,1600",
    "--remote-debugging-address=127.0.0.1",
    "--remote-debugging-port=0",   # let Chrome pick
    "--remote-allow-origins=*",
    "--disable-crashpad",          # <-- critical
    "--disable-breakpad",          # <-- critical
    "--noerrdialogs",
    "--hide-scrollbars",
    "--mute-audio",
]
log_path = "/mnt/data/chrome_stderr.log"
with open(log_path, "wb") as err:
    chrome_proc = subprocess.Popen(
        chrome_cmd,
        stdout=subprocess.DEVNULL,
        stderr=err,
        env={**os.environ},
    )

# 4) Wait for DevToolsActivePort and read the chosen port
active_port_file = os.path.join(profile_dir, "DevToolsActivePort")

def wait_for_active_port(filepath, timeout=25):
    start = time.time()
    while time.time() - start < timeout:
        if os.path.exists(filepath):
            try:
                with open(filepath, "r") as f:
                    lines = f.read().strip().splitlines()
                if lines:
                    return int(lines[0].strip())
            except Exception:
                pass
        time.sleep(0.2)
    return None

port = wait_for_active_port(active_port_file, timeout=25)
if not port:
    try:
        with open(log_path, "rb") as f:
            tail = f.read()[-6144:]
        print("Chrome stderr tail:\n", tail.decode("utf-8", errors="ignore"))
    except Exception:
        pass
    chrome_proc.terminate()
    raise RuntimeError("Chrome did not produce DevToolsActivePort; could not determine debug port.")

print(f"Chrome chose port: {port}")

# 5) Attach Selenium to the running Chrome
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

def make_driver_attached(port: int):
    opts = Options()
    opts.add_experimental_option("debuggerAddress", f"127.0.0.1:{port}")
    service = Service(executable_path=DRIVER_BIN)
    drv = webdriver.Chrome(service=service, options=opts)
    drv._chrome_proc = chrome_proc
    drv._profile_dir = profile_dir
    return drv

def cleanup_attached_driver(drv):
    try: drv.quit()
    except: pass
    try:
        if hasattr(drv, "_chrome_proc"): drv._chrome_proc.terminate()
    except: pass
    try:
        if hasattr(drv, "_profile_dir"): shutil.rmtree(drv._profile_dir, ignore_errors=True)
    except: pass

# 6) Sanity test
drv = make_driver_attached(port)
drv.get("https://example.com")
print("Sanity title:", drv.title)  # expect: Example Domain
cleanup_attached_driver(drv)
print("OK ✅  (profile:", profile_dir, ")")


Chrome chose port: 45345
Sanity title: Example Domain
OK ✅  (profile: /mnt/data/chrome-prof-c0c150aeaf454eaaa4a7f31fe6a76055 )


In [ ]:
# Start headless Chrome (fresh profile), read DevToolsActivePort, and attach Selenium.
# Returns: driver, cleanup()
import os, subprocess, shutil, time, uuid, stat

CHROME_DIR = "/usr/local/chrome-linux64"
CHROME_BIN = f"{CHROME_DIR}/chrome"
DRIVER_BIN = "/usr/local/chromedriver-linux64/chromedriver"

def start_attached_chrome():
    # permissions (crashpad etc.)
    for helper in ("chrome_crashpad_handler", "chrome_sandbox", "nacl_helper", "nacl_helper_bootstrap"):
        p = os.path.join(CHROME_DIR, helper)
        if os.path.exists(p):
            os.chmod(p, os.stat(p).st_mode | stat.S_IEXEC)
    os.chmod(CHROME_BIN, os.stat(CHROME_BIN).st_mode | stat.S_IEXEC)

    # fresh HOME + fresh profile
    home_dir    = f"/mnt/data/home-{uuid.uuid4().hex}"
    profile_dir = f"/mnt/data/chrome-prof-{uuid.uuid4().hex}"
    os.makedirs(home_dir, exist_ok=True)
    os.makedirs(profile_dir, exist_ok=True)
    env = {**os.environ, "HOME": home_dir}

    # spawn chrome (let it pick port)
    chrome_cmd = [
        CHROME_BIN,
        "--headless=new",
        f"--user-data-dir={profile_dir}",
        "--no-sandbox",
        "--disable-dev-shm-usage",
        "--disable-gpu",
        "--disable-extensions",
        "--disable-notifications",
        "--no-first-run",
        "--no-default-browser-check",
        "--disable-background-networking",
        "--disable-sync",
        "--metrics-recording-only",
        "--safebrowsing-disable-auto-update",
        "--window-size=1280,1600",
        "--remote-debugging-address=127.0.0.1",
        "--remote-debugging-port=0",
        "--remote-allow-origins=*",
        "--disable-crashpad",
        "--disable-breakpad",
        "--noerrdialogs",
        "--hide-scrollbars",
        "--mute-audio",
    ]
    log_path = "/mnt/data/chrome_stderr.log"
    with open(log_path, "wb") as err:
        chrome_proc = subprocess.Popen(chrome_cmd, stdout=subprocess.DEVNULL, stderr=err, env=env)

    # wait for DevToolsActivePort
    active_port_file = os.path.join(profile_dir, "DevToolsActivePort")
    deadline = time.time() + 25
    port = None
    while time.time() < deadline:
        if os.path.exists(active_port_file):
            try:
                with open(active_port_file, "r") as f:
                    lines = f.read().strip().splitlines()
                if lines:
                    port = int(lines[0].strip())
                    break
            except:
                pass
        time.sleep(0.2)
    if not port:
        try:
            with open(log_path, "rb") as f:
                print(f.read()[-6144:].decode("utf-8", "ignore"))
        except: pass
        chrome_proc.terminate()
        raise RuntimeError("Chrome didn't provide DevToolsActivePort")

    # attach Selenium
    from selenium import webdriver
    from selenium.webdriver.chrome.service import Service
    from selenium.webdriver.chrome.options import Options

    opts = Options()
    opts.add_experimental_option("debuggerAddress", f"127.0.0.1:{port}")
    service = Service(executable_path=DRIVER_BIN)
    driver = webdriver.Chrome(service=service, options=opts)

    def cleanup():
        try: driver.quit()
        except: pass
        try: chrome_proc.terminate()
        except: pass
        try: shutil.rmtree(profile_dir, ignore_errors=True)
        except: pass

    return driver, cleanup


In [ ]:
# Deps
!pip -q install --upgrade selenium beautifulsoup4 python-dotenv openai
import os, time, json, csv
from typing import List, Dict
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# ---- Config ----
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") or "sk-Yourapikey"
assert os.getenv("OPENAI_API_KEY"), "Set OPENAI_API_KEY."

TARGET_URL = "https://forums.edmunds.com/discussion/50806/general/x/car-subscription-vs-lease-vs-purchase"
MAX_PAGES  = 1          # start with 1; increase after success
WAIT_SECS  = 12
SCROLL_PAUSE = 0.35

OUTPUT_CSV  = "final data.csv"
CSV_HEADERS = ["Date", "user_id", "comments"]

# ---- Page helpers ----
def wait_ready(driver):
    WebDriverWait(driver, WAIT_SECS).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "div.Comment"))
    )

def extract_page_records(driver) -> List[Dict]:
    soup = BeautifulSoup(driver.page_source, "html.parser")
    rows = []
    for block in soup.select("div.Comment"):
        author_el = block.select_one(".Author") or block.select_one("a.Username")
        time_el   = block.select_one("time")
        body_el   = block.select_one(".Item-Body") or block.select_one(".Message") or block

        rows.append({
            "author":          (author_el.get_text(strip=True) if author_el else ""),
            "timestamp_text":  (time_el.get_text(strip=True) if time_el else ""),
            "timestamp_title": (time_el.get("title","") if time_el else ""),
            "comment_text":    (body_el.get_text(" ", strip=True) if body_el else ""),
            "comment_html":    (str(body_el) if body_el else "")
        })
    return rows

def crawl_raw(base_url: str = TARGET_URL, max_pages: int = MAX_PAGES) -> List[Dict]:
    driver, cleanup = start_attached_chrome()
    out: List[Dict] = []
    try:
        for i in range(1, max_pages+1):
            url = base_url if i == 1 else f"{base_url.rstrip('/')}/p{i}"
            print("Visiting:", url)
            driver.get(url)
            wait_ready(driver)

            # lazy-load scroll
            last_h = 0
            while True:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(SCROLL_PAUSE)
                h = driver.execute_script("return document.body.scrollHeight")
                if h == last_h: break
                last_h = h

            rows = extract_page_records(driver)
            for r in rows:
                r["page_url"] = url
            print("  Extracted:", len(rows))
            out.extend(rows)
    finally:
        cleanup()
    print("Total raw records:", len(out))
    return out

# ---- Prompt shaping (day5-style) -> exact CSV schema ----
try:
    from openai import OpenAI
    _STYLE = "sdk_v1"
except Exception:
    import openai as OpenAI  # type: ignore
    _STYLE = "legacy"

MODEL = os.getenv("OPENAI_MODEL", "gpt-4o-mini")

SYSTEM_PROMPT = """
You transform forum comment records into a CSV-ready schema used by an existing pipeline.
Your output must be JSON with key "items" mapping to an array of objects.
Each object MUST have exactly these keys and casing:
- Date
- user_id
- comments

Mapping rules (strict):
- "Date": use timestamp_title if non-empty; otherwise use timestamp_text. Do not reformat; return as-is.
- "user_id": copy author as-is.
- "comments": a cleaned version of comment_text:
    • Remove quoted previous messages and signatures.
    • Keep only the author's current message content.
    • Preserve natural punctuation and spacing.
    • Do not invent content.

Never add extra keys. If a source field is empty, set the corresponding value to "".
Return only JSON, no markdown fences.
"""

def llm_client():
    if _STYLE == "sdk_v1":
        client = OpenAI()
        client.api_key = os.getenv("OPENAI_API_KEY")
        return client
    else:
        import openai
        openai.api_key = os.getenv("OPENAI_API_KEY")
        return openai

def prompt_to_exact_rows(records: List[Dict], model: str = MODEL, batch_size: int = 25) -> List[Dict]:
    client = llm_client()
    out: List[Dict] = []
    for i in range(0, len(records), batch_size):
        chunk = records[i:i+batch_size]
        payload = [{"author": r.get("author",""),
                    "timestamp_text": r.get("timestamp_text",""),
                    "timestamp_title": r.get("timestamp_title",""),
                    "comment_text": r.get("comment_text","")} for r in chunk]

        if _STYLE == "sdk_v1":
            resp = client.chat.completions.create(
                model=model,
                messages=[
                    {"role": "system", "content": SYSTEM_PROMPT},
                    {"role": "user", "content": json.dumps({"items": payload})}
                ],
                response_format={"type": "json_object"},
                temperature=0
            )
            content = resp.choices[0].message.content
        else:
            resp = OpenAI.ChatCompletion.create(
                model=model,
                messages=[
                    {"role": "system", "content": SYSTEM_PROMPT},
                    {"role": "user", "content": json.dumps({"items": payload})}
                ],
                temperature=0
            )
            content = resp.choices[0].message["content"]

        try:
            data = json.loads(content)
            for row in data.get("items", []):
                out.append({
                    "Date": row.get("Date",""),
                    "user_id": row.get("user_id",""),
                    "comments": row.get("comments","")
                })
        except Exception as e:
            print("JSON parse error (fallback passthrough):", e)
            for r in chunk:
                out.append({
                    "Date": r.get("timestamp_title") or r.get("timestamp_text") or "",
                    "user_id": r.get("author",""),
                    "comments": r.get("comment_text","")
                })
    return out

def write_exact_csv(rows: List[Dict], path: str = OUTPUT_CSV):
    with open(path, "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=CSV_HEADERS)
        writer.writeheader()
        for r in rows:
            writer.writerow({
                "Date":    r.get("Date",""),
                "user_id": r.get("user_id",""),
                "comments": r.get("comments","")
            })
    print(f"Wrote {path} with {len(rows)} rows.")

def run_pipeline_exact():
    raw  = crawl_raw(TARGET_URL, MAX_PAGES)
    rows = prompt_to_exact_rows(raw, model=MODEL)
    write_exact_csv(rows)

# 🚀 Run it (start with 1 page)
run_pipeline_exact()

# Peek first rows
import pandas as pd
display(pd.read_csv(OUTPUT_CSV).head())


Visiting: https://forums.edmunds.com/discussion/50806/general/x/car-subscription-vs-lease-vs-purchase
  Extracted: 50
Total raw records: 50
Wrote final data.csv with 50 rows.


,Date,user_id,comments
0,"April 7, 2018 11:15AM",biancar,Zero interest in the car subscription idea. I ...
1,"April 7, 2018 11:38AM",Mr_Shiftright,I hadn't thought about being forced into anoth...
2,"April 7, 2018 1:37PM",rkelly17,Also here in PA tax is on the monthly payment....
3,"April 20, 2018 7:11AM",PF_Flyer,I don't see what the consumer gains in this ar...
4,"April 20, 2018 8:47AM",stickguy,"price aside, it is the convenience they are se..."


In [ ]:
from google.colab import files
files.download("final data.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# ===== IMDB reviews config =====
IMDB_URL = "https://www.imdb.com/title/tt4154796/reviews"  # Avengers: Endgame; change to any title's /reviews
REVIEW_LIMIT = 50  # hard cap as requested

import os
assert os.getenv("OPENAI_API_KEY"), "Set OPENAI_API_KEY first (same as your agent)."

# Optional: choose a small, cheap model; your agent used 'gpt-4o-mini'
MODEL = os.getenv("OPENAI_MODEL", "gpt-4o-mini")


In [ ]:
# ===== Hardened IMDB reviews scraper helpers =====
import time, re
from typing import List, Dict
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

IMDB_WAIT = 25  # longer wait for first load

def _click_if_present(driver, how, sel, timeout=3) -> bool:
    try:
        btn = WebDriverWait(driver, timeout).until(EC.element_to_be_clickable((how, sel)))
        driver.execute_script("arguments[0].click();", btn)
        return True
    except Exception:
        return False

def _handle_imdb_consent(driver):
    # Try several common consent/accept buttons IMDB uses in different regions/builds
    selectors = [
        (By.ID, "onetrust-accept-btn-handler"),                  # OneTrust cookie banner
        (By.CSS_SELECTOR, "button[aria-label*='Agree' i]"),
        (By.CSS_SELECTOR, "button[aria-label*='Accept' i]"),
        (By.CSS_SELECTOR, "button#cookie-consent-accept"),       # legacy
        (By.CSS_SELECTOR, "button.ipc-btn[aria-label*='Accept' i]"),
    ]
    for how, sel in selectors:
        if _click_if_present(driver, how, sel, timeout=2):
            time.sleep(0.5)  # let overlay disappear
            break

def _wait_reviews_any(driver, timeout=IMDB_WAIT):
    # Wait until we see any container that can hold reviews
    candidates = [
        (By.CSS_SELECTOR, "div.lister-list div.review-container"),
        (By.CSS_SELECTOR, "div.review-container"),
        (By.CSS_SELECTOR, "section.lister-list"),
        (By.CSS_SELECTOR, "div.lister-item-content"),
    ]
    WebDriverWait(driver, timeout).until(
        EC.presence_of_element_located(candidates[0]) or
        EC.presence_of_element_located(candidates[1]) or
        EC.presence_of_element_located(candidates[2]) or
        EC.presence_of_element_located(candidates[3])
    )

def _extract_current_reviews(driver) -> List[Dict]:
    soup = BeautifulSoup(driver.page_source, "html.parser")
    # Grab both legacy and new containers
    containers = soup.select("div.review-container")
    if not containers:
        containers = soup.select("div.lister-list div.review-container, div.lister-item-content")

    out = []
    for c in containers:
        # author
        a = c.select_one(".display-name-link a, a.ipc-link.ipc-link--base")
        author = a.get_text(strip=True) if a else ""
        # date
        d = c.select_one(".review-date, .ipc-link.ipc-link--secondary-text")
        date = d.get_text(strip=True) if d else ""
        # title (not used for CSV but handy)
        t = c.select_one(".title, a.review-title, h3, h3.ipc-title__text")
        title = t.get_text(" ", strip=True) if t else ""
        # rating
        r = c.select_one(".rating-other-user-rating span, span.ipc-rating-star--rating")
        rating = r.get_text(strip=True) if r else ""
        # body text (many variants)
        body = c.select_one("div.text.show-more__control, div.content div.text, .ipc-html-content, .text")
        body_text = body.get_text(" ", strip=True) if body else ""
        if any([author, date, body_text]):
            out.append({
                "author": author,
                "date": date,
                "rating": rating,
                "title": title,
                "comment_text": body_text
            })
    return out

def _try_click_load_more(driver) -> bool:
    """
    Try multiple IMDB 'Load more' buttons; return True if clicked.
    """
    candidates = [
        (By.CSS_SELECTOR, "button.ipc-btn[aria-label*='Load More' i]"),
        (By.CSS_SELECTOR, "button.ipl-load-more__button"),
        (By.ID, "load-more-trigger"),
        (By.CSS_SELECTOR, "button.ipc-see-more__button"),
        (By.XPATH, "//button[contains(., 'Load more') or contains(., 'Load More')]"),
    ]
    for how, sel in candidates:
        try:
            btn = WebDriverWait(driver, 4).until(EC.element_to_be_clickable((how, sel)))
            driver.execute_script("arguments[0].scrollIntoView({block:'center'});", btn)
            time.sleep(0.2)
            driver.execute_script("arguments[0].click();", btn)
            return True
        except Exception:
            pass
    return False

def imdb_collect_reviews(url: str, limit: int = 50) -> List[Dict]:
    driver, cleanup = start_attached_chrome()  # uses your verified launcher
    try:
        driver.get(url)
        _handle_imdb_consent(driver)
        _wait_reviews_any(driver, timeout=IMDB_WAIT)

        all_rows = []
        seen = set()
        # Loop until we hit limit or run out of "Load more"
        while True:
            time.sleep(0.8)  # give time for incremental loads
            rows = _extract_current_reviews(driver)
            # de-dup by (author, title, date, first 60 chars)
            for r in rows:
                k = (r.get("author",""), r.get("title",""), r.get("date",""), r.get("comment_text","")[:60])
                if k not in seen:
                    seen.add(k)
                    all_rows.append(r)
            if len(all_rows) >= limit:
                break
            if not _try_click_load_more(driver):
                break
        return all_rows[:limit]
    finally:
        cleanup()


In [ ]:
# ===== Prompt cleaning to strict CSV schema =====
import json

try:
    from openai import OpenAI
    _STYLE = "sdk_v1"
except Exception:
    import openai as OpenAI  # type: ignore
    _STYLE = "legacy"

SYSTEM_PROMPT_IMDB = """
You transform movie review records into a CSV-ready schema used by an existing pipeline.
Return JSON with key "items" mapping to an array of objects. Each object MUST have exactly:
- Date
- user_id
- comments

Mapping rules:
- "Date": copy the review's date string as-is (do NOT reformat).
- "user_id": copy the author display name as-is.
- "comments": a cleaned version of the review text:
    • Remove spoiler warnings, "Permalink"/"Was this helpful?" lines, and UI boilerplate.
    • Keep only the reviewer’s actual content.
    • Preserve natural punctuation and spacing.
    • DO NOT invent or summarize; just clean.
If a source field is empty, set the value to "".
Return only JSON (no markdown fences).
"""

def llm_client():
    if _STYLE == "sdk_v1":
        c = OpenAI()
        c.api_key = os.getenv("OPENAI_API_KEY")
        return c
    else:
        import openai
        openai.api_key = os.getenv("OPENAI_API_KEY")
        return openai

def prompt_clean_imdb(records: List[Dict], model: str = MODEL, batch_size: int = 25) -> List[Dict]:
    client = llm_client()
    out = []
    for i in range(0, len(records), batch_size):
        chunk = records[i:i+batch_size]
        payload = [{
            "author": r.get("author",""),
            "date": r.get("date",""),
            "comment_text": r.get("comment_text","")
        } for r in chunk]

        if _STYLE == "sdk_v1":
            resp = client.chat.completions.create(
                model=model,
                messages=[
                    {"role":"system","content":SYSTEM_PROMPT_IMDB},
                    {"role":"user","content":json.dumps({"items": payload})}
                ],
                response_format={"type":"json_object"},
                temperature=0
            )
            content = resp.choices[0].message.content
        else:
            resp = OpenAI.ChatCompletion.create(
                model=model,
                messages=[
                    {"role":"system","content":SYSTEM_PROMPT_IMDB},
                    {"role":"user","content":json.dumps({"items": payload})}
                ],
                temperature=0
            )
            content = resp.choices[0].message["content"]

        try:
            data = json.loads(content)
            for row in data.get("items", []):
                out.append({
                    "Date": row.get("Date",""),
                    "user_id": row.get("user_id",""),
                    "comments": row.get("comments","")
                })
        except Exception as e:
            # Fallback passthrough (keeps pipeline running)
            for r in chunk:
                out.append({
                    "Date": r.get("date",""),
                    "user_id": r.get("author",""),
                    "comments": r.get("comment_text","")
                })
    return out


In [ ]:
# Stealthier Chrome attach for IMDB: UA spoof + webdriver=false + misc tweaks
import os, subprocess, shutil, time, uuid, stat

CHROME_DIR = "/usr/local/chrome-linux64"
CHROME_BIN = f"{CHROME_DIR}/chrome"
DRIVER_BIN = "/usr/local/chromedriver-linux64/chromedriver"

IMDB_UA = ("Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 "
           "(KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36")
IMDB_LANG = "en-US,en;q=0.9"

def start_attached_chrome_imdb():
    # permissions (for crashpad etc.)
    for helper in ("chrome_crashpad_handler","chrome_sandbox","nacl_helper","nacl_helper_bootstrap"):
        p = os.path.join(CHROME_DIR, helper)
        if os.path.exists(p):
            os.chmod(p, os.stat(p).st_mode | stat.S_IEXEC)
    os.chmod(CHROME_BIN, os.stat(CHROME_BIN).st_mode | stat.S_IEXEC)

    home_dir    = f"/mnt/data/home-{uuid.uuid4().hex}"
    profile_dir = f"/mnt/data/chrome-prof-{uuid.uuid4().hex}"
    os.makedirs(home_dir, exist_ok=True)
    os.makedirs(profile_dir, exist_ok=True)
    env = {**os.environ, "HOME": home_dir}

    cmd = [
        CHROME_BIN,
        "--headless=new",
        f"--user-data-dir={profile_dir}",
        "--no-sandbox",
        "--disable-dev-shm-usage",
        "--disable-gpu",
        "--disable-extensions",
        "--no-first-run",
        "--no-default-browser-check",
        "--disable-background-networking",
        "--disable-sync",
        "--metrics-recording-only",
        "--safebrowsing-disable-auto-update",
        "--window-size=1280,2000",
        "--remote-debugging-address=127.0.0.1",
        "--remote-debugging-port=0",
        "--remote-allow-origins=*",
        "--disable-crashpad",
        "--disable-breakpad",
        f"--user-agent={IMDB_UA}",
        f"--lang={IMDB_LANG}",
        "--disable-blink-features=AutomationControlled",
    ]
    log_path = "/mnt/data/chrome_stderr.log"
    with open(log_path, "wb") as err:
        chrome_proc = subprocess.Popen(cmd, stdout=subprocess.DEVNULL, stderr=err, env=env)

    # Wait for DevToolsActivePort
    active = os.path.join(profile_dir, "DevToolsActivePort")
    deadline = time.time() + 25
    port = None
    while time.time() < deadline:
        if os.path.exists(active):
            try:
                port = int(open(active).read().strip().splitlines()[0])
                break
            except:
                pass
        time.sleep(0.2)
    if not port:
        chrome_proc.terminate()
        raise RuntimeError("Chrome didn't provide DevToolsActivePort")

    # Attach Selenium and add stealth CDP patches
    from selenium import webdriver
    from selenium.webdriver.chrome.service import Service
    from selenium.webdriver.chrome.options import Options

    opts = Options()
    opts.add_experimental_option("debuggerAddress", f"127.0.0.1:{port}")
    service = Service(executable_path=DRIVER_BIN)
    driver = webdriver.Chrome(service=service, options=opts)

    # Stealth patches: navigator.webdriver=false, proper UA/Language
    driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
        "source": """
          Object.defineProperty(navigator, 'webdriver', {get: () => undefined});
          Object.defineProperty(navigator, 'languages', {get: () => ['en-US','en']});
          Object.defineProperty(navigator, 'platform', {get: () => 'Linux x86_64'});
        """
    })
    driver.execute_cdp_cmd("Network.setUserAgentOverride", {
        "userAgent": IMDB_UA,
        "acceptLanguage": IMDB_LANG,
        "platform": "Linux"
    })

    def cleanup():
        try: driver.quit()
        except: pass
        try: chrome_proc.terminate()
        except: pass
        try: shutil.rmtree(profile_dir, ignore_errors=True)
        except: pass

    return driver, cleanup


In [ ]:
# Hardened IMDB review collection with consent handling and mobile fallback
import time, re
from typing import List, Dict
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

IMDB_WAIT = 35   # give it more time

def _click_first(driver, candidates, timeout=4) -> bool:
    for how, sel in candidates:
        try:
            el = WebDriverWait(driver, timeout).until(EC.element_to_be_clickable((how, sel)))
            driver.execute_script("arguments[0].click();", el)
            time.sleep(0.4)
            return True
        except Exception:
            pass
    return False

def _handle_imdb_consent(driver):
    # Try multiple possible consent buttons
    cand = [
        (By.ID, "onetrust-accept-btn-handler"),
        (By.CSS_SELECTOR, "button[aria-label*='Agree' i]"),
        (By.CSS_SELECTOR, "button[aria-label*='Accept' i]"),
        (By.CSS_SELECTOR, "button.ipc-btn[aria-label*='Accept' i]"),
        (By.CSS_SELECTOR, "button#cookie-consent-accept"),
    ]
    _click_first(driver, cand, timeout=3)

def _wait_any_reviews(driver, timeout=IMDB_WAIT) -> bool:
    try:
        WebDriverWait(driver, timeout).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "div.review-container, div.lister-list div.review-container, .lister-list, .ipc-list"))
        )
        return True
    except Exception:
        return False

def _extract_current_reviews(driver) -> List[Dict]:
    soup = BeautifulSoup(driver.page_source, "html.parser")
    containers = soup.select("div.review-container")
    if not containers:
        containers = soup.select("div.lister-list div.review-container, div.lister-item-content")
    if not containers:
        containers = soup.select(".ipc-list .ipc-list-card__content, .ipc-list-card__content")  # mobile-ish blocks

    out = []
    for c in containers:
        author = ""
        date = ""
        rating = ""
        title = ""
        body_text = ""

        a = c.select_one(".display-name-link a, a.ipc-link.ipc-link--base, a.author")
        if a: author = a.get_text(strip=True)

        d = c.select_one(".review-date, .ipc-link.ipc-link--secondary-text, span.review-date, .content .actions time")
        if d: date = d.get_text(strip=True)

        t = c.select_one(".title, a.review-title, h3, h3.ipc-title__text")
        if t: title = t.get_text(" ", strip=True)

        r = c.select_one(".rating-other-user-rating span, span.ipc-rating-star--rating")
        if r: rating = r.get_text(strip=True)

        b = c.select_one("div.text.show-more__control, div.content div.text, .ipc-html-content, .text, .content .text")
        if b: body_text = b.get_text(" ", strip=True)

        if any([author, date, body_text]):
            out.append({
                "author": author,
                "date": date,
                "rating": rating,
                "title": title,
                "comment_text": body_text
            })
    return out

def _try_click_load_more(driver) -> bool:
    cand = [
        (By.CSS_SELECTOR, "button.ipc-btn[aria-label*='Load More' i]"),
        (By.CSS_SELECTOR, "button.ipl-load-more__button"),
        (By.ID, "load-more-trigger"),
        (By.CSS_SELECTOR, "button.ipc-see-more__button"),
        (By.XPATH, "//button[contains(., 'Load more') or contains(., 'Load More')]"),
    ]
    return _click_first(driver, cand, timeout=5)

def imdb_collect_reviews_hardened(url: str, limit: int = 50) -> List[Dict]:
    driver, cleanup = start_attached_chrome_imdb()
    try:
        # Try desktop first (sorted by helpfulness to keep deterministic items)
        if "reviews" not in url:
            url = url.rstrip("/") + "/reviews"
        if "sort=" not in url:
            url = url + ("&" if "?" in url else "?") + "sort=helpfulnessScore&dir=desc&ratingFilter=0"

        driver.get(url)
        _handle_imdb_consent(driver)
        ok = _wait_any_reviews(driver, timeout=IMDB_WAIT)

        # Fallback to mobile reviews page if desktop layout fails
        if not ok or "Our systems have detected unusual traffic" in driver.page_source:
            m_url = url.replace("www.imdb.com", "m.imdb.com").replace("/reviews", "/reviews")
            driver.get(m_url)
            _handle_imdb_consent(driver)
            _wait_any_reviews(driver, timeout=IMDB_WAIT)

        seen, all_rows = set(), []
        while True:
            time.sleep(1.0)
            rows = _extract_current_reviews(driver)
            for r in rows:
                k = (r.get("author",""), r.get("title",""), r.get("date",""), r.get("comment_text","")[:80])
                if k not in seen:
                    seen.add(k)
                    all_rows.append(r)
            if len(all_rows) >= limit:
                break
            if not _try_click_load_more(driver):
                break

        return all_rows[:limit]
    finally:
        cleanup()


In [ ]:
# Hardened IMDB review collection with consent handling and mobile fallback
import time, re
from typing import List, Dict
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

IMDB_WAIT = 35   # give it more time

def _click_first(driver, candidates, timeout=4) -> bool:
    for how, sel in candidates:
        try:
            el = WebDriverWait(driver, timeout).until(EC.element_to_be_clickable((how, sel)))
            driver.execute_script("arguments[0].click();", el)
            time.sleep(0.4)
            return True
        except Exception:
            pass
    return False

def _handle_imdb_consent(driver):
    # Try multiple possible consent buttons
    cand = [
        (By.ID, "onetrust-accept-btn-handler"),
        (By.CSS_SELECTOR, "button[aria-label*='Agree' i]"),
        (By.CSS_SELECTOR, "button[aria-label*='Accept' i]"),
        (By.CSS_SELECTOR, "button.ipc-btn[aria-label*='Accept' i]"),
        (By.CSS_SELECTOR, "button#cookie-consent-accept"),
    ]
    _click_first(driver, cand, timeout=3)

def _wait_any_reviews(driver, timeout=IMDB_WAIT) -> bool:
    try:
        WebDriverWait(driver, timeout).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "div.review-container, div.lister-list div.review-container, .lister-list, .ipc-list"))
        )
        return True
    except Exception:
        return False

def _extract_current_reviews(driver) -> List[Dict]:
    soup = BeautifulSoup(driver.page_source, "html.parser")
    containers = soup.select("div.review-container")
    if not containers:
        containers = soup.select("div.lister-list div.review-container, div.lister-item-content")
    if not containers:
        containers = soup.select(".ipc-list .ipc-list-card__content, .ipc-list-card__content")  # mobile-ish blocks

    out = []
    for c in containers:
        author = ""
        date = ""
        rating = ""
        title = ""
        body_text = ""

        a = c.select_one(".display-name-link a, a.ipc-link.ipc-link--base, a.author")
        if a: author = a.get_text(strip=True)

        d = c.select_one(".review-date, .ipc-link.ipc-link--secondary-text, span.review-date, .content .actions time")
        if d: date = d.get_text(strip=True)

        t = c.select_one(".title, a.review-title, h3, h3.ipc-title__text")
        if t: title = t.get_text(" ", strip=True)

        r = c.select_one(".rating-other-user-rating span, span.ipc-rating-star--rating")
        if r: rating = r.get_text(strip=True)

        b = c.select_one("div.text.show-more__control, div.content div.text, .ipc-html-content, .text, .content .text")
        if b: body_text = b.get_text(" ", strip=True)

        if any([author, date, body_text]):
            out.append({
                "author": author,
                "date": date,
                "rating": rating,
                "title": title,
                "comment_text": body_text
            })
    return out

def _try_click_load_more(driver) -> bool:
    cand = [
        (By.CSS_SELECTOR, "button.ipc-btn[aria-label*='Load More' i]"),
        (By.CSS_SELECTOR, "button.ipl-load-more__button"),
        (By.ID, "load-more-trigger"),
        (By.CSS_SELECTOR, "button.ipc-see-more__button"),
        (By.XPATH, "//button[contains(., 'Load more') or contains(., 'Load More')]"),
    ]
    return _click_first(driver, cand, timeout=5)

def imdb_collect_reviews_hardened(url: str, limit: int = 50) -> List[Dict]:
    driver, cleanup = start_attached_chrome_imdb()
    try:
        # Try desktop first (sorted by helpfulness to keep deterministic items)
        if "reviews" not in url:
            url = url.rstrip("/") + "/reviews"
        if "sort=" not in url:
            url = url + ("&" if "?" in url else "?") + "sort=helpfulnessScore&dir=desc&ratingFilter=0"

        driver.get(url)
        _handle_imdb_consent(driver)
        ok = _wait_any_reviews(driver, timeout=IMDB_WAIT)

        # Fallback to mobile reviews page if desktop layout fails
        if not ok or "Our systems have detected unusual traffic" in driver.page_source:
            m_url = url.replace("www.imdb.com", "m.imdb.com").replace("/reviews", "/reviews")
            driver.get(m_url)
            _handle_imdb_consent(driver)
            _wait_any_reviews(driver, timeout=IMDB_WAIT)

        seen, all_rows = set(), []
        while True:
            time.sleep(1.0)
            rows = _extract_current_reviews(driver)
            for r in rows:
                k = (r.get("author",""), r.get("title",""), r.get("date",""), r.get("comment_text","")[:80])
                if k not in seen:
                    seen.add(k)
                    all_rows.append(r)
            if len(all_rows) >= limit:
                break
            if not _try_click_load_more(driver):
                break

        return all_rows[:limit]
    finally:
        cleanup()


In [ ]:
# Run hardened IMDb pipeline
raw_reviews = imdb_collect_reviews_hardened(IMDB_URL, REVIEW_LIMIT)
print(f"Collected raw reviews: {len(raw_reviews)}")

import pandas as pd, csv
raw_df = pd.DataFrame([{
    "Date": r.get("date",""),
    "user_id": r.get("author",""),
    "rating": r.get("rating",""),
    "title": r.get("title",""),
    "comments_raw": r.get("comment_text","")
} for r in raw_reviews])
display(raw_df.head(10))

# Clean via your prompt step (reusing prompt_clean_imdb from earlier)
clean_rows = prompt_clean_imdb(raw_reviews, model=MODEL)
clean_df = pd.DataFrame(clean_rows)
display(clean_df.head(10))

# Diff
cmp = raw_df.merge(clean_df, on=["Date","user_id"], how="left")
cmp["raw_len"]   = cmp["comments_raw"].fillna("").str.len()
cmp["clean_len"] = cmp["comments"].fillna("").str.len()
cmp["changed"]   = cmp["comments_raw"].fillna("") != cmp["comments"].fillna("")
cmp["delta_len"] = cmp["clean_len"] - cmp["raw_len"]
print("=== Difference summary ===")
print(cmp["changed"].value_counts())
display(cmp[["Date","user_id","raw_len","clean_len","delta_len","changed"]].head(10))

# Save and download
CSV_PATH = "imdb_reviews_cleaned.csv"
with open(CSV_PATH, "w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=["Date","user_id","comments"])
    writer.writeheader()
    for r in clean_rows:
        writer.writerow(r)
print(f"Saved: {CSV_PATH}  (rows={len(clean_rows)})")

from google.colab import files
files.download(CSV_PATH)


Collected raw reviews: 50


,Date,user_id,rating,title,comments_raw
0,,,8,Endgame,There isn't too much in the way of suspense or...
1,,,9,The zenith of the MCU franchise.,I feel like I'm wasting my time writing down m...
2,,,10,They had no right to do that,First review from me. This film deserves it. A...
3,,,10,Crazy in every sense,This film is an emotional rollercoaster with s...
4,,,10,Epic Ending,"Thank you Marvel, End Game is an ending that g..."
5,,,10,Perfect ending,"After watching Infinity war, I was looking for..."
6,,,10,Epic movie,As always avengers didn't disappoint even with...
7,,,10,The ending made all 22 movies worth it,"If you're going to watch this movie, avoid any..."
8,,,10,The End of an Era!,"After Avengers Infinity War, we waited for the..."
9,,,6,So much potential lost ....,After Infinity War I had huge expectations. Un...


,Date,user_id,comments
0,,,There isn't too much in the way of suspense or...
1,,,I feel like I'm wasting my time writing down m...
2,,,First review from me. This film deserves it. A...
3,,,This film is an emotional rollercoaster with s...
4,,,"Thank you Marvel, End Game is an ending that g..."
5,,,"After watching Infinity war, I was looking for..."
6,,,As always avengers didn't disappoint even with...
7,,,"If you're going to watch this movie, avoid any..."
8,,,"After Avengers Infinity War, we waited for the..."
9,,,After Infinity War I had huge expectations. Un...


=== Difference summary ===
changed
True     2466
False      34
Name: count, dtype: int64


,Date,user_id,raw_len,clean_len,delta_len,changed
0,,,549,547,-2,True
1,,,549,2049,1500,True
2,,,549,975,426,True
3,,,549,324,-225,True
4,,,549,64,-485,True
5,,,549,98,-451,True
6,,,549,144,-405,True
7,,,549,537,-12,True
8,,,549,2718,2169,True
9,,,549,659,110,True


Saved: imdb_reviews_cleaned.csv  (rows=50)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd

# In Colab, the file should be in the current working directory
csv_path = "imdb_reviews_cleaned.csv"

df = pd.read_csv(csv_path)

print(df.shape)
df.head()



(50, 3)


,Date,user_id,comments
0,NaN,NaN,There isn't too much in the way of suspense or...
1,NaN,NaN,I feel like I'm wasting my time writing down m...
2,NaN,NaN,First review from me. This film deserves it. A...
3,NaN,NaN,This film is an emotional rollercoaster with s...
4,NaN,NaN,"Thank you Marvel, End Game is an ending that g..."


In [ ]:
from collections import Counter
import re

# Very rough keyword buckets
positives = ["great", "amazing", "fantastic", "emotional", "love", "epic", "thank", "best"]
negatives = ["slow", "boring", "waste", "predictable", "bad", "worst", "disappoint", "confusing"]

likes, dislikes = [], []

for text in df["comments"].dropna():
    low = text.lower()
    if any(w in low for w in positives):
        likes.append(text)
    if any(w in low for w in negatives):
        dislikes.append(text)

print(f"Found {len(likes)} positive-leaning reviews")
print(f"Found {len(dislikes)} negative-leaning reviews")




Found 28 positive-leaning reviews
Found 13 negative-leaning reviews


In [ ]:
from openai import OpenAI
client = OpenAI()

prompt = f"""
You are an analyst. Summarize key themes from IMDb reviews.

### Likes (positive mentions):
{likes[:25]}  # sample to avoid token overload

### Dislikes (negative mentions):
{dislikes[:25]}

Write a clear report with:
- What users liked (top themes)
- What can be improved (top criticisms)
- One-sentence conclusion
"""

resp = client.chat.completions.create(
    model="gpt-4o-mini",  # light model works fine
    messages=[{"role":"user","content":prompt}],
    temperature=0.4
)

print(resp.choices[0].message.content)


### Report on IMDb Reviews of Avengers: Endgame

#### What Users Liked (Top Themes):
1. **Emotional Impact**: Many viewers praised the film for its emotional depth, noting that it elicited strong feelings, including laughter and tears, while successfully paying tribute to a decade of Marvel films.
2. **Nostalgia and Fan Service**: The film was lauded for its references and callbacks to previous Marvel movies, which resonated well with long-time fans and added layers of enjoyment.
3. **Visual Spectacle**: Reviewers highlighted the impressive visual effects and epic battle sequences, describing them as some of the best in the Marvel franchise.
4. **Character Development**: The film was appreciated for its focus on the original Avengers, particularly Iron Man, Captain America, and Thor, showcasing their growth and providing a satisfying conclusion to their arcs.
5. **Engaging Storytelling**: Despite some criticisms, many found the plot inventive and engaging, with a well-paced narrative t

In [ ]:
report = resp.choices[0].message.content
with open("imdb_review_report.txt","w",encoding="utf-8") as f:
    f.write(report)

from google.colab import files
files.download("imdb_review_report.txt")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# === IMDb Review Report -> PDF (Colab) ===
# Produces imdb_review_report.pdf and downloads it.

!pip -q install reportlab openai pandas numpy

import os, re, math, textwrap, random
import pandas as pd
import numpy as np
from collections import Counter, defaultdict
from reportlab.lib.pagesizes import LETTER
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, PageBreak, Table, TableStyle
from reportlab.lib import colors
from reportlab.lib.units import inch
from reportlab.lib.enums import TA_LEFT
from datetime import datetime

# --------- CONFIG ---------
CSV_PATH = "imdb_reviews_cleaned.csv"     # adjust if needed
PDF_PATH = "imdb_review_report.pdf"
TITLE    = "IMDb Review Insights"
SUBTITLE = "What Users Liked & What Can Be Improved"
USE_LLM  = True                           # set to False to skip LLM summary
MODEL    = os.getenv("OPENAI_MODEL", "gpt-4o-mini")

# --------- Load data ---------
df = pd.read_csv(CSV_PATH)
assert {"Date","user_id","comments"}.issubset(df.columns), "CSV must have Date, user_id, comments"

# Drop empties
df["comments"] = df["comments"].fillna("").astype(str)
df = df[df["comments"].str.strip()!=""].copy()
n_reviews = len(df)

# --------- Quick heuristics: sentiment-ish buckets ---------
pos_words = {"great","amazing","fantastic","emotional","love","epic","brilliant","excellent","powerful","fun","masterpiece","enjoy","best","heart"}
neg_words = {"slow","boring","waste","predictable","bad","worst","disappoint","confusing","weak","long","overrated","cliché","drag","mess","cringe"}

def bucket_review(txt: str):
    low = txt.lower()
    p = sum(1 for w in pos_words if w in low)
    n = sum(1 for w in neg_words if w in low)
    if p>n: return "positive"
    if n>p: return "negative"
    return "neutral"

df["bucket"] = df["comments"].apply(bucket_review)

# sample examples
def sample_texts(d, bucket, k=3):
    subset = d[d["bucket"]==bucket]["comments"].tolist()
    random.shuffle(subset)
    return subset[:k]

pos_examples = sample_texts(df, "positive", 3)
neg_examples = sample_texts(df, "negative", 3)

# top words (very light cleaning)
stop = set("""
a an and the to is it in of for on with as this that was are be but have has had from by about or not at
i we you they he she them his her our their my your me us its it's
film movie movies one can will just more also even if so too very
""".split())
word_counts = Counter()
for t in df["comments"]:
    words = re.findall(r"[a-zA-Z']{3,}", t.lower())
    word_counts.update([w for w in words if w not in stop])
top_terms = [f"{w} ({c})" for w,c in word_counts.most_common(12)]

# --------- Optional LLM summary ---------
llm_likes, llm_improve, llm_conclusion = None, None, None
if USE_LLM and os.getenv("OPENAI_API_KEY"):
    try:
        from openai import OpenAI
        client = OpenAI()
        # Take a sample of reviews to keep the prompt small
        sample_for_llm = df["comments"].sample(min(60, n_reviews), random_state=42).tolist()

        sys_prompt = (
            "You are a sharp product analyst. Given a sample of IMDb movie reviews, "
            "extract concise bullet lists: (1) what users liked, (2) what can be improved. "
            "Avoid spoilers and plot details. Focus on experience and quality signals. "
            "Keep bullets crisp (5-8 bullets each). Then give a one-sentence overall conclusion."
        )
        user_prompt = "Sample reviews:\n" + "\n---\n".join(sample_for_llm)

        resp = client.chat.completions.create(
            model=MODEL,
            messages=[
                {"role":"system","content":sys_prompt},
                {"role":"user","content":user_prompt}
            ],
            temperature=0.3
        ).choices[0].message.content

        # naive split
        # Expect sections like "What users liked:", "What can be improved:", "Conclusion:"
        liked_block = re.search(r"(?:What users liked|Likes)[:\n]+([\s\S]*?)(?:What can be improved|Improvements|Conclusion|$)", resp, re.I)
        improve_block = re.search(r"(?:What can be improved|Improvements)[:\n]+([\s\S]*?)(?:Conclusion|$)", resp, re.I)
        conclusion_block = re.search(r"(?:Conclusion)[:\n]+([\s\S]*)$", resp, re.I)

        llm_likes = (liked_block.group(1).strip() if liked_block else None)
        llm_improve = (improve_block.group(1).strip() if improve_block else None)
        llm_conclusion = (conclusion_block.group(1).strip() if conclusion_block else None)
    except Exception as e:
        llm_likes = llm_improve = llm_conclusion = None

# --------- Fallback heuristics if no LLM ---------
def bullets_from_examples(examples, positive=True):
    if not examples: return ["(no clear examples found)"]
    hints = []
    for t in examples:
        low = t.lower()
        if positive:
            if "acting" in low: hints.append("Strong performances/acting.")
            if "story" in low or "plot" in low: hints.append("Story resonated with many viewers.")
            if "music" in low or "score" in low: hints.append("Music/score praised.")
            if "visual" in low or "effects" in low or "cgi" in low: hints.append("Visuals/effects appreciated.")
            if "emotional" in low or "tears" in low or "cry" in low: hints.append("Emotional impact landed.")
        else:
            if "slow" in low or "long" in low or "drag" in low: hints.append("Pacing felt slow/overlong for some.")
            if "predictable" in low or "clich" in low: hints.append("Plot felt predictable/clichéd.")
            if "confus" in low: hints.append("Some viewers found parts confusing.")
            if "character" in low: hints.append("Character development could be stronger.")
            if "dialog" in low: hints.append("Dialogue/lines didn’t always work.")
    # dedupe, cap
    out = []
    for h in hints:
        if h not in out:
            out.append(h)
        if len(out) >= 8:
            break
    if not out:
        out = ["General positive reception." if positive else "General areas for improvement noted."]
    return out

if not llm_likes:
    llm_likes = "\n".join(f"- {b}" for b in bullets_from_examples(pos_examples, True))
if not llm_improve:
    llm_improve = "\n".join(f"- {b}" for b in bullets_from_examples(neg_examples, False))
if not llm_conclusion:
    llm_conclusion = "Overall, reviewers highlight clear strengths while noting a few consistent areas to improve."

# --------- Build the PDF ---------
styles = getSampleStyleSheet()
H1 = styles["Heading1"]; H1.alignment = TA_LEFT
H2 = styles["Heading2"]; H2.alignment = TA_LEFT
Body = styles["BodyText"]; Body.leading = 14

def as_paras(text):
    lines = [ln for ln in text.splitlines() if ln.strip()!=""]
    return [Paragraph(ln, Body) for ln in lines]

doc = SimpleDocTemplate(
    PDF_PATH,
    pagesize=LETTER,
    rightMargin=36, leftMargin=36, topMargin=36, bottomMargin=36,
    title=TITLE, author="ScraperAgent"
)

flow = []
flow.append(Paragraph(TITLE, H1))
flow.append(Paragraph(SUBTITLE, H2))
flow.append(Paragraph(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M')} • Reviews analyzed: {n_reviews}", Body))
flow.append(Spacer(1, 10))

# Overview stats
pos_n = int((df["bucket"]=="positive").sum())
neg_n = int((df["bucket"]=="negative").sum())
neu_n = n_reviews - pos_n - neg_n
flow.append(Paragraph("<b>Overview</b>", H2))
flow += as_paras(f"""
• Positive-leaning: {pos_n}
• Negative-leaning: {neg_n}
• Neutral: {neu_n}
""")
flow.append(Spacer(1, 8))
flow.append(Paragraph("<b>Top Frequent Terms</b>", H2))
flow += as_paras("• " + ", ".join(top_terms))
flow.append(Spacer(1, 12))

# Likes / Improvements
flow.append(Paragraph("<b>What Users Liked</b>", H2))
flow += as_paras(llm_likes)
flow.append(Spacer(1, 8))

flow.append(Paragraph("<b>What Can Be Improved</b>", H2))
flow += as_paras(llm_improve)
flow.append(Spacer(1, 12))

# Examples
def add_examples(title, items):
    flow.append(Paragraph(title, H2))
    if not items:
        flow.append(Paragraph("(no examples found)", Body))
        return
    for i, ex in enumerate(items, 1):
        flow.append(Paragraph(f"<b>Example {i}:</b> {textwrap.shorten(ex, width=900, placeholder='…')}", Body))
        flow.append(Spacer(1, 6))

add_examples("Positive Review Snippets", pos_examples)
flow.append(Spacer(1, 6))
add_examples("Negative Review Snippets", neg_examples)
flow.append(Spacer(1, 12))

# Conclusion
flow.append(Paragraph("<b>Conclusion</b>", H2))
flow += as_paras(llm_conclusion)

# Appendix
flow.append(PageBreak())
flow.append(Paragraph("Appendix: Method (ScraperAgent)", H2))
flow += as_paras("""
• Collected IMDb reviews via Selenium, cleaned to a strict schema (Date, user_id, comments).
• Bucketed reviews with simple keyword heuristics for a quick signal.
• Summarized themes with an LLM (if enabled) to express insights clearly.
• This report is for demonstration/education; not an official IMDb analysis.
""")

doc.build(flow)

print(f"Saved PDF → {PDF_PATH}")

# Offer download (Colab)
from google.colab import files
files.download(PDF_PATH)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 33.1 MB/s eta 0:00:00
Saved PDF → imdb_review_report.pdf


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!git config --global user.name "sukratishm"
!git config --global user.email "sukrati.95@example.com"


In [ ]:
!git clone https://github.com/sukratishm/ScraperAgent.git


Cloning into 'ScraperAgent'...


In [ ]:
!ls /content


'final data.csv'	  imdb_review_report.txt     sample_data
 imdb_review_report.pdf   imdb_reviews_cleaned.csv   ScraperAgent


In [ ]:
!file /content/ScraperAgent



/content/ScraperAgent: directory


In [ ]:
!ls /content



'final data.csv'	  imdb_reviews_cleaned.csv  'ScraperAgent (3).ipynb'
 imdb_review_report.pdf   sample_data
 imdb_review_report.txt   ScraperAgent
